## Importing Libraries

In [121]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
import requests
import folium

## Getting the Url

In [122]:
link = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(link, 'lxml')


In [123]:
#Get the html source
html = link
df = pd.read_html(html)[0]


In [124]:
df.columns = ['Postcode', 'Borough', 'Neighbourhood']
df.drop(0, inplace = True)

In [125]:
df.reset_index(inplace = True)
df.drop(['index'], axis = 1, inplace = True)

In [126]:
df.drop(list(df[df['Borough'] == 'Not assigned'].index), axis = 0, inplace = True)
df.reset_index(inplace = True)

## Placing copied postcodes on the same line, dropping extra columns

In [127]:
 for i in range(len(df) - 1):
        if df['Postcode'][i] == df['Postcode'][i+1]:
            df['Neighbourhood'][i] = df['Neighbourhood'][i] + ', ' + df['Neighbourhood'][i+1]
            df['Borough'][i + 1] = 'copy'
        
        
df.drop('index', inplace = True, axis = 1)
df.drop(list(df[df['Borough'] == 'copy'].index), inplace = True)
df.reset_index(inplace = True)  
df[df['Borough'] == df['Neighbourhood']]


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,index,Postcode,Borough,Neighbourhood


In [128]:
df.drop('index', inplace = True, axis = 1)

In [129]:
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Not assigned


In [130]:
df.shape



(103, 3)

## Getting Geographical Data

In [131]:
geo = pd.read_csv('http://cocl.us/Geospatial_data')
geo.head(n = 20)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [132]:
df = df.merge(geo, how = 'inner', left_on = 'Postcode', right_on = 'Postal Code')

In [133]:
df.drop('Postcode',axis = 1, inplace = True)

## Foursquare Log In

In [150]:
CLIENT_ID = 'CZA01IYKO51C3OOV0VN1CCDEKYMQSELTEK3CMTL5STBL01FN' # your Foursquare ID
CLIENT_SECRET = 'PLLFNP1TD10Z5EDNHXV0LOXKMF13CDMBFCENSNSTTJHPJGN0' # your Foursquare Secret
VERSION = '20181220'

## Group Data with Neighborhoods in North York

In [151]:
dfny = df[df['Borough'] == 'North York'].reset_index(drop = True)

In [152]:
dfny.head()

,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,North York,Parkwoods,M3A,43.753259,-79.329656
1,North York,Victoria Village,M4A,43.725882,-79.315572
2,North York,"Lawrence Heights, Lawrence Manor",M6A,43.718518,-79.464763
3,North York,Don Mills North,M3B,43.745906,-79.352188
4,North York,Glencairn,M6B,43.709577,-79.445073


## Put North York Into a Map

In [153]:
addy_north = 'North York,Toronto'
latitude_north = 43.753259
longitude_north = -79.43
print('The geograpical coordinate of North York are {}, {}.'.format(latitude_north, longitude_north))

The geograpical coordinate of North York are 43.753259, -79.43.


In [154]:
map_north = folium.Map(location=[latitude_north, longitude_north], zoom_start=11.5)

# add markers to map
for lat, lng, label in zip(dfny['Latitude'], dfny['Longitude'], dfny['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_north)  
    
map_north

## Find the top 50 Venues in Parkwoods, North York

In [155]:
neighborhood_latitude = dfny.loc[0, 'Latitude'] # neighbourhood latitude value
neighborhood_longitude = dfny.loc[0, 'Longitude'] # neighbourhood longitude value

neighborhood_name = dfny.loc[0, 'Neighbourhood'] # neighbourhood name

print('Latitude and longitude values of "{}" are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of "Parkwoods" are 43.7532586, -79.3296565.


In [162]:
LIMIT = 50
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_north, longitude_north, VERSION, radius, LIMIT)

In [163]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c1c4d69351e3d5f4e0d86af'},
 'response': {'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 43.7577590045, 'lng': -79.42378173980772},
   'sw': {'lat': 43.7487589955, 'lng': -79.43621826019229}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4d9e237f7958f04df15426fa',
       'name': 'Earl Bales Ski and Snowboard Centre',
       'location': {'address': '4169 Bathurst St.',
        'crossStreet': 'Raoul Wallenberg Rd.',
        'lat': 43.75263075010656,
        'lng': -79.4318654481669,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.75263075010656,
          'lng': -79.4318654481669}],
        'distance': 

In [164]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [165]:
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

venues = results['response']['groups'][0]['items']  
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(10)

,name,categories,lat,lng
0,Earl Bales Ski and Snowboard Centre,Ski Chalet,43.752631,-79.431865
1,North York Ski Centre,Ski Area,43.752711,-79.431953
2,Earl Bales Dog Park,Dog Run,43.755700,-79.430873
3,Earl Bales Community Centre,Community Center,43.754447,-79.434910


In [166]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))


4 venues were returned by Foursquare.


In [167]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

## Get venues for each neighborhood in North York

In [170]:
north_venues = getNearbyVenues(names=dfny['Neighbourhood'],
                                   latitudes=dfny['Latitude'],
                                   longitudes=dfny['Longitude']
                                  )

Parkwoods
Victoria Village
Lawrence Heights, Lawrence Manor
Don Mills North
Glencairn
Flemingdon Park, Don Mills South
Hillcrest Village
Bathurst Manor, Downsview North
Fairview, Henry Farm
Northwood Park, York University
Bayview Village
CFB Toronto, Downsview East
Silver Hills, York Mills
Downsview West
Maple Leaf Park, North Park
Humber Summit
Newtonbrook, Willowdale
Downsview Central
Bedford Park, Lawrence Manor East
Emery, Humberlea
Willowdale South
Downsview Northwest
York Mills West
Willowdale West


In [171]:
north_venues.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
5,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
6,Victoria Village,43.725882,-79.315572,Eglinton Ave E & Sloane Ave/Bermondsey Rd,43.726086,-79.313620,Intersection
7,Victoria Village,43.725882,-79.315572,Pizza Nova,43.725824,-79.312860,Pizza Place
8,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,Roots,43.718476,-79.466869,Boutique
9,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,Lac Vien Vietnamese Restaurant,43.721259,-79.468472,Vietnamese Restaurant


In [172]:
north_venues.groupby('Neighborhood').count()


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor, Downsview North",17,17,17,17,17,17
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",24,24,24,24,24,24
"CFB Toronto, Downsview East",3,3,3,3,3,3
Don Mills North,5,5,5,5,5,5
Downsview Central,4,4,4,4,4,4
Downsview Northwest,4,4,4,4,4,4
Downsview West,4,4,4,4,4,4
"Emery, Humberlea",2,2,2,2,2,2


In [173]:
print('There are {} uniques categories.'.format(len(north_venues['Venue Category'].unique())))


There are 104 uniques categories.


In [174]:

# one hot encoding
north_onehot = pd.get_dummies(north_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
north_onehot['Neighborhood'] = north_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [north_onehot.columns[-1]] + list(north_onehot.columns[:-1])
north_onehot = north_onehot[fixed_columns]

north_onehot.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,...,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [175]:
north_onehot.shape


(226, 105)

In [176]:
north_grouped = north_onehot.groupby('Neighborhood').mean().reset_index()
north_grouped.head(7)

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,...,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store
0,"Bathurst Manor, Downsview North",0.0,0.000000,0.000000,0.0,0.0,0.00,0.0,0.058824,0.0,...,0.058824,0.0,0.000000,0.0,0.0,0.0,0.058824,0.0,0.0,0.0
1,Bayview Village,0.0,0.000000,0.000000,0.0,0.0,0.00,0.0,0.250000,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,"Bedford Park, Lawrence Manor East",0.0,0.000000,0.041667,0.0,0.0,0.00,0.0,0.000000,0.0,...,0.041667,0.0,0.041667,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
3,"CFB Toronto, Downsview East",0.0,0.333333,0.000000,0.0,0.0,0.00,0.0,0.000000,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
4,Don Mills North,0.0,0.000000,0.000000,0.0,0.0,0.00,0.0,0.000000,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
5,Downsview Central,0.0,0.000000,0.000000,0.0,0.0,0.00,0.0,0.000000,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
6,Downsview Northwest,0.0,0.000000,0.000000,0.0,0.0,0.25,0.0,0.000000,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


In [177]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [178]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = north_grouped['Neighborhood']

for ind in np.arange(north_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(north_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Downsview North",Coffee Shop,Frozen Yogurt Shop,Fast Food Restaurant,Deli / Bodega,Pharmacy,Pizza Place,Diner,Bridal Shop,Restaurant,Chinese Restaurant
1,Bayview Village,Japanese Restaurant,Chinese Restaurant,Café,Bank,Event Space,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant,Diner
2,"Bedford Park, Lawrence Manor East",Fast Food Restaurant,Italian Restaurant,Coffee Shop,Grocery Store,Restaurant,Greek Restaurant,Hardware Store,Hobby Shop,Indian Restaurant,Juice Bar
3,"CFB Toronto, Downsview East",Airport,Bus Stop,Park,Women's Store,Electronics Store,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega,Department Store
4,Don Mills North,Japanese Restaurant,Gym / Fitness Center,Caribbean Restaurant,Café,Basketball Court,Event Space,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant
5,Downsview Central,Home Service,Food Truck,Business Service,Baseball Field,Women's Store,Empanada Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega,Department Store
6,Downsview Northwest,Grocery Store,Gym / Fitness Center,Athletics & Sports,Discount Store,Event Space,Construction & Landscaping,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant
7,Downsview West,Grocery Store,Bank,Moving Target,Shopping Mall,Empanada Restaurant,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega,Department Store
8,"Emery, Humberlea",Construction & Landscaping,Baseball Field,Women's Store,Event Space,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant,Diner,Discount Store
9,"Fairview, Henry Farm",Clothing Store,Fast Food Restaurant,Coffee Shop,Toy / Game Store,Shoe Store,Candy Store,Movie Theater,Food Court,Burrito Place,Men's Store


In [180]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

north_data = dfny.drop(16)
# set number of clusters
kclusters = 5

north_grouped_clustering = north_grouped.drop('Neighborhood', 1)


# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(north_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 
#len(kmeans.labels_)#=16
#north_data.shape

array([0, 0, 0, 0, 0, 0, 0, 0, 4, 0], dtype=int32)

In [181]:
north_merged = north_data

# add clustering labels
north_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
north_merged = north_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

north_merged

,Borough,Neighbourhood,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,Parkwoods,M3A,43.753259,-79.329656,0,Park,Food & Drink Shop,Fast Food Restaurant,Electronics Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega,Department Store
1,North York,Victoria Village,M4A,43.725882,-79.315572,0,Coffee Shop,Pizza Place,Hockey Arena,Portuguese Restaurant,Intersection,Electronics Store,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega
2,North York,"Lawrence Heights, Lawrence Manor",M6A,43.718518,-79.464763,0,Clothing Store,Furniture / Home Store,Accessories Store,Fraternity House,Miscellaneous Shop,Boutique,Event Space,Coffee Shop,Vietnamese Restaurant,Arts & Crafts Store
3,North York,Don Mills North,M3B,43.745906,-79.352188,0,Japanese Restaurant,Gym / Fitness Center,Caribbean Restaurant,Café,Basketball Court,Event Space,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant
4,North York,Glencairn,M6B,43.709577,-79.445073,0,Italian Restaurant,Japanese Restaurant,Metro Station,Bakery,Pub,Construction & Landscaping,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant
5,North York,"Flemingdon Park, Don Mills South",M3C,43.725900,-79.340923,0,Beer Store,Coffee Shop,Gym,Asian Restaurant,Grocery Store,Clothing Store,Chinese Restaurant,General Entertainment,Dim Sum Restaurant,Japanese Restaurant
6,North York,Hillcrest Village,M2H,43.803762,-79.363452,0,Golf Course,Pool,Mediterranean Restaurant,Fast Food Restaurant,Dog Run,Electronics Store,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega
7,North York,"Bathurst Manor, Downsview North",M3H,43.754328,-79.442259,0,Coffee Shop,Frozen Yogurt Shop,Fast Food Restaurant,Deli / Bodega,Pharmacy,Pizza Place,Diner,Bridal Shop,Restaurant,Chinese Restaurant
8,North York,"Fairview, Henry Farm",M2J,43.778517,-79.346556,4,Clothing Store,Fast Food Restaurant,Coffee Shop,Toy / Game Store,Shoe Store,Candy Store,Movie Theater,Food Court,Burrito Place,Men's Store
9,North York,"Northwood Park, York University",M3J,43.767980,-79.487262,0,Furniture / Home Store,Miscellaneous Shop,Bar,Massage Studio,Coffee Shop,Women's Store,Electronics Store,Construction & Landscaping,Cosmetics Shop,Deli / Bodega


In [182]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location = [latitude_north, longitude_north], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(north_merged['Latitude'], north_merged['Longitude'], north_merged['Neighbourhood'], north_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## The End, we have succesfully run KMEANs and grouped the clusters
